**BERTTWEET STANCE DETECTION - Immigration**


Alkan Can Mollaibrahimoglu CSS MA Koç University




### Importing Necessary Libraries

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

### Data Loading

In [ ]:
df=pd.read_excel('/content/stance_data_6.xlsx')

In [ ]:
df=df.drop_duplicates(subset=['_id'])

In [ ]:
df=df[['Tweet','stance']]
df

,Tweet,stance
0,zurnanın zırt dediği yerde tamda burası bugüne...,0
1,kusura bakmada bundan yıl önce çocukken bende ...,2
2,irkçılık cezalandırılmağı suya karışmış zehir ...,1
3,çocuk fabrikası kurdunuz dabizim haberimiz mi ...,2
4,ırkçısınız sığınmacı düşmanısınız faşistsiniz ...,2
...,...,...
1149,defolun gidin lan pislikler ülkenizde bulamadı...,2
1150,sizirahatasizediyoruz köprü altından sonra sığ...,1
1151,rt gönlümüz ruhumuz dualarımız ekmeğimiz karde...,1
1152,rt maraş merkezde mülteci̇ bi̇ ai̇le i̇çi̇n ev...,1


In [ ]:
df= df.dropna(subset=['Tweet', 'stance'])


In [ ]:
df['stance'].value_counts()

,count
stance,
0,476
2,454
1,222
?,2


### Sampling

In [ ]:
df_positive=df[df['stance']==1]

In [ ]:
df_negative=df[df['stance']==2]

In [ ]:
df_neutral=df[df['stance']==0]

In [ ]:
df_negative

,Tweet,stance
1,kusura bakmada bundan yıl önce çocukken bende ...,2
3,çocuk fabrikası kurdunuz dabizim haberimiz mi ...,2
4,ırkçısınız sığınmacı düşmanısınız faşistsiniz ...,2
5,kadar gocmen geldiki dogal artik siginmacilar ...,2
6,geçen gün kayboldum yolu suriyeliler tarif ett...,2
...,...,...
1145,burda sürünüyorlar reziller dönün faydanız ols...,2
1146,zaten yerel seçimlerde zafere vermek mantıklıs...,2
1147,bir ak partili olarak mayısta reise verdiğim o...,2
1148,kardeşimben memleketimin sahillerinde nargile ...,2


In [ ]:
df_negative=df_negative.sample(n=444,random_state=5)

In [ ]:
df_neutral=df_neutral.sample(n=444,random_state=5)

### Train Test Split (%60-%20-%20)




In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Combine the relevant and irrelevant dataframes
df_combined = pd.concat([df_negative, df_positive, df_neutral])

X_train, X_temp, y_train, y_temp = train_test_split(
    df_combined['Tweet'],  # Tweets as input
    df_combined['stance'],  # Use 'stance' as the target
    test_size=0.4,  # 40% for the temporary set
    random_state=40,  # Ensuring reproducibility
    stratify=df_combined['stance']  # Stratify split based on the 'stance'
)

# Split the temporary data into validation and test sets (50% each of temp data)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.5,  # 50% of temp data to each of validation and test sets
    random_state=40,  # Ensuring reproducibility
    stratify=y_temp  # Stratify split based on the 'stance'
)

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer
import random
import numpy as np
def set_seed(seed_value=42):
    """Set seed for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)  # if you are using GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


In [ ]:
set_seed(42)  #

### Verification

In [ ]:
print(f"Training Set: {X_train.shape[0]} tweets")
print(f"Testing Set: {X_test.shape[0]} tweets")
print(f"Validation Set: {X_val.shape[0]} tweets")

Training Set: 666 tweets
Testing Set: 222 tweets
Validation Set: 222 tweets


In [ ]:
y_train.value_counts()

,count
stance,
0,251
2,251
1,125


In [ ]:
y_test.value_counts()

label
0    138
1     69
Name: count, dtype: int64

In [ ]:
y_val.value_counts()

label
0    138
1     69
Name: count, dtype: int64

### BERTweet Stance

In [ ]:
X_test.isna().any()

Tweet    False
dtype: bool

In [ ]:
X_train.isna().any()



Tweet    False
dtype: bool

In [ ]:
X_val.isna().any()

Tweet    False
dtype: bool

In [ ]:
X_train

1203     yine rezil rezil top oynuyolar avrupayi geçtim...
734      hak özgürlükler mücadelesinin değil ülkenin ya...
1419     chpli mersin mezitli belediyesi ilçenin suriye...
5491     saat itibariylesuriyeli leredirneden doyran kö...
345      rt vatanını seviyorsan kararver on milyon mült...
                               ...                        
257                           arap müritlerini al git bari
4777     türkiyeyi göçmenistan yaptınız para icin sattı...
11474    milyon nüfuslu nijeryada vaka sayısı haftalık ...
7419     hakanınşifası anneşefkatiallah anne babasına b...
11696          suriyelileriistemiyoruz ameka lami cimi yok
Name: Tweet, Length: 414, dtype: object

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from torch.utils.data import Dataset
import torch
import numpy as np
from sklearn.metrics import f1_score

# Define the compute_metrics function for F1 score
def compute_metrics(p):
    predictions = np.argmax(p.predictions, axis=1)
    return {"f1": f1_score(p.label_ids, predictions, average='macro')}  # Using 'macro' for multi-class

def tokenize_data(X, tokenizer):
    return tokenizer(X.tolist(), padding=True, truncation=True, max_length=128, return_tensors='pt')

class TweetDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Load tokenizer and model
model_name = "VRLLab/TurkishBERTweet"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)  # Three stances


train_encodings = tokenize_data(X_train, tokenizer)
val_encodings = tokenize_data(X_val, tokenizer)
test_encodings = tokenize_data(X_test, tokenizer)

train_dataset = TweetDataset(train_encodings, y_train.tolist())
val_dataset = TweetDataset(val_encodings, y_val.tolist())
test_dataset = TweetDataset(test_encodings, y_test.tolist())
# Define a dataset class


# Example usage and training as previously outlined

training_args = TrainingArguments(

    output_dir='./results',            # Output directory
    evaluation_strategy="epoch",       # Evaluation is done at the end of each epoch
    save_strategy="epoch",             # Save model at the end of each epoch
    learning_rate=2e-5,                # Defines the learning rate
    per_device_train_batch_size=8,     # Batch size for training
    per_device_eval_batch_size=16,     # Batch size for evaluation
    num_train_epochs=3,                # Number of training epochs
    weight_decay=0.01,                 # Weight decay for regularization
    load_best_model_at_end=True,       # Load the best model when finished training
    metric_for_best_model='f1'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()
test_results = trainer.evaluate(test_dataset)
print(test_results)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.88M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.77M [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/652M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at VRLLab/TurkishBERTweet and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-17-e9a57ecbe584>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,F1
1,No log,0.935396,0.553100
2,No log,0.989409,0.588453
3,No log,1.061202,0.635628


<ipython-input-17-e9a57ecbe584>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-17-e9a57ecbe584>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-17-e9a57ecbe584>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-17-e9a57ecbe584>:21: UserWarning: To copy construct fro

{'eval_loss': 0.8941858410835266, 'eval_f1': 0.6990415493735275, 'eval_runtime': 40.3017, 'eval_samples_per_second': 5.508, 'eval_steps_per_second': 0.347, 'epoch': 3.0}


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Assuming you have already trained your model and the `trainer` is set up
# Evaluate the model on the test set
test_results = trainer.predict(test_dataset)  # Returns an object with predictions
predicted_labels = np.argmax(test_results.predictions, axis=1)

# Print classification report
# Assuming y_test is available and contains the true labels from the test set
print("Classification Report:")
print(classification_report(test_dataset.labels, predicted_labels))



<ipython-input-17-e9a57ecbe584>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.81      0.75        89
           1       0.76      0.49      0.59        45
           2       0.74      0.76      0.75        88

    accuracy                           0.73       222
   macro avg       0.73      0.69      0.70       222
weighted avg       0.73      0.73      0.72       222

